In [18]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [19]:
#verileri oku
from basemodels.models import FikstureModelData
import pandas as pd

data = FikstureModelData.objects.all().values()
data_list = list(data)
df = pd.DataFrame(data_list)
df

,id,data_id,tarih,data,count,isprogress,created_at,updated_at
0,2,20241021,2024-10-21,"[{'tournament': {'name': 'Premier League', 'sl...",333,False,2024-10-22 11:50:27.834100+00:00,2024-10-22 11:50:27.834100+00:00
1,3,20241022,2024-10-22,[{'tournament': {'name': 'UEFA Champions Leagu...,221,False,2024-10-22 11:59:31.073050+00:00,2024-10-22 11:59:31.073050+00:00


In [20]:
from basemodels.models import Tournament, Season, Team
from basemodels.models import FikstureModelData


class SofascoreAPI:
    def __init__(self):
        pass

    def fetch_data(self):
        data = FikstureModelData.objects.all().values()
        data_list = list(data)
        df = pd.DataFrame(data_list)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        veri = expanded_data.copy()
        veri.columns = [item.replace(".", "_") for item in veri.columns]
        return veri

    def tournament_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("tournament_uniqueTournament_id")
        
        cols = [
            "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
            "tournament_category_slug", "tournament_uniqueTournament_category_id", 
            "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
            "tournament_uniqueTournament_hasPerformanceGraphFeature"
        ]
        
        for col in cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[cols]

        for _, row in verim.iterrows():
            tournament_id = row['tournament_uniqueTournament_id']
            
            if pd.isna(tournament_id):
                print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                continue
        
            if not Tournament.objects.filter(tournament_id=tournament_id).exists():
                Tournament.objects.create(
                    tournament_id=row['tournament_uniqueTournament_id'],
                    tournament_name=row['tournament_name'],
                    tournament_slug=row['tournament_slug'],
                    tournament_category_name=row['tournament_category_name'],
                    tournament_category_slug=row['tournament_category_slug'],
                    tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                    tournament_model_id=row['tournament_id'],
                    tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                    tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                )
                print(f"Tournament ID {tournament_id} başarıyla eklendi.")
            else:
                print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")

    def season_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        verim = verim.drop_duplicates("season_id")
        
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')
    
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            if not Season.objects.filter(season_id=season_id).exists():
                tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
                
                if tournament_instance:
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance  # Doğru sütun adı burada kullanılıyor
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")
    
   
    def team_data(self):

        data = self.fetch_data()
        homecols = [
                    "homeTeam_id", "homeTeam_name", "homeTeam_slug", "homeTeam_shortName",
                    "homeTeam_nameCode", "homeTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        awaycols = [
                    "awayTeam_id", "awayTeam_name", "awayTeam_slug", "awayTeam_shortName",
                    "awayTeam_nameCode", "awayTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        # Home takımı için veri seçimi ve kolon isimlerinin ayarlanması
        homedata = data[homecols]
        homedata.columns = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Away takımı için veri seçimi ve kolon isimlerinin ayarlanması
        awaydata = data[awaycols]
        awaydata.columns  = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Home ve Away takımları birleştirme
        teamdata = pd.concat([homedata, awaydata])
        teamdata = teamdata.drop_duplicates(subset=["team_id", "tournament_id", "season_id"])
       
        for _, row in teamdata.iterrows():
            team_id = row['team_id']
            tournament_id = row['tournament_id']
            season_id = row['season_id']
            
            # Geçersiz veya eksik id'leri atla
            if pd.isna(team_id) or pd.isna(tournament_id) or pd.isna(season_id):
                print(f"Geçersiz değer: Team ID={team_id}, Tournament ID={tournament_id}, Season ID={season_id}. Atlanıyor.")
                continue

            # İlgili tournament ve season nesnelerini getir
            tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()

            if tournament_instance and season_instance:
                # Takımın veritabanında var olup olmadığını kontrol et, yoksa oluştur
                if not Team.objects.filter(team_id=team_id, tournament=tournament_instance, season=season_instance).exists():
                    Team.objects.create(
                        team_id=team_id,
                        team_name=row['team_name'],
                        team_slug=row['team_slug'],
                        team_shortName=row['team_shortName'],
                        team_nameCode=row['team_nameCode'],
                        team_national=row['team_national'],
                        tournament=tournament_instance,  # ForeignKey olan alanlar
                        season=season_instance           # ForeignKey olan alanlar
                    )
                    print(f"Team ID {team_id} başarıyla eklendi.")
                else:
                    print(f"Team ID {team_id} zaten mevcut, eklenmedi.")
            else:
                print(f"Tournament ID {tournament_id} veya Season ID {season_id} bulunamadı.")


# SofascoreAPI sınıfını kullanarak team_data'yı veritabanına kaydetmek için:
api = SofascoreAPI()
teamdata = api.fetch_data()  # Veriyi getirin (API'den ya da dosyadan)
api.tournament_data()
api.season_data()
api.team_data()

print("İşlemler bitti")


Tournament ID 17 zaten mevcut, eklenmedi.
Tournament ID 8 zaten mevcut, eklenmedi.
Tournament ID 23 zaten mevcut, eklenmedi.
Tournament ID 35 zaten mevcut, eklenmedi.
Tournament ID 34 zaten mevcut, eklenmedi.
Tournament ID 37 zaten mevcut, eklenmedi.
Tournament ID 325 zaten mevcut, eklenmedi.
Tournament ID 52 zaten mevcut, eklenmedi.
Tournament ID 38 zaten mevcut, eklenmedi.
Tournament ID 185 zaten mevcut, eklenmedi.
Tournament ID 2132 zaten mevcut, eklenmedi.
Tournament ID 155 zaten mevcut, eklenmedi.
Tournament ID 11621 zaten mevcut, eklenmedi.
Tournament ID 170 zaten mevcut, eklenmedi.
Tournament ID 210 zaten mevcut, eklenmedi.
Tournament ID 18 zaten mevcut, eklenmedi.
Tournament ID 54 zaten mevcut, eklenmedi.
Tournament ID 53 zaten mevcut, eklenmedi.
Tournament ID 182 zaten mevcut, eklenmedi.
Tournament ID 390 zaten mevcut, eklenmedi.
Tournament ID 131 zaten mevcut, eklenmedi.
Tournament ID 98 zaten mevcut, eklenmedi.
Tournament ID 45 zaten mevcut, eklenmedi.
Tournament ID 266 zate

In [8]:
Seasonx = Season.objects.all().values()
Seasonx = list(Seasonx)
Seasonx = pd.DataFrame(Seasonx)
Seasonx

,season_id,season_name,season_year,tournament_id
0,57284,Allsvenskan 2024,2024,40
1,57318,USL League One 2024,2024,13362
2,57322,Eliteserien 2024,2024,20
3,57356,1st Division 2024,2024,22
4,57359,"Toppserien, Women 2024",2024,201
...,...,...,...,...
99,65687,Primera Division Women 24/25,24/25,1127
100,65961,Indian Super League 24/25,24/25,1900
101,66456,UEFA Youth League 24/25,24/25,2132
102,66662,Super League 2 24/25,24/25,186


In [9]:
Teamdata = Team.objects.all().values()
Teamdata = list(Teamdata)
Teamdata= pd.DataFrame(Teamdata)
Teamdata

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id,season_id
0,3,Wolverhampton,wolverhampton,Wolves,WOL,False,17,61627
1,7,Crystal Palace,crystal-palace,Crystal Palace,CRY,False,17,61627
2,14,Nottingham Forest,nottingham-forest,Forest,NFO,False,17,61627
3,17,Manchester City,manchester-city,Man City,MCI,False,17,61627
4,38,Chelsea,chelsea,Chelsea,CHE,False,17,61627
...,...,...,...,...,...,...,...,...
654,826802,Fiorentina U20,acf-fiorentina-u20-u20,Fiorentina U20,ACF,False,2292,64432
655,826810,Udinese U20,udinese-calcio-u20-u20,Udinese U20,UDI,False,2292,64432
656,826812,Genoa U20,genoa-cfc-u20-u20,Genoa U20,GEN,False,2292,64432
657,826817,Sampdoria U20,sampdoria-genoa-u20-u20,Sampdoria U20,SAM,False,2292,64432


In [15]:
takım = Teamdata[Teamdata["team_id"]==3052]
takım

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id,season_id
200,3052,Fenerbahçe,fenerbahce,Fenerbahçe,FEN,False,52,63814


In [17]:
from basemodels.models import Tournament, Season, Team
Tournamentx = Tournament.objects.all().values()
Tournamentx = list(Tournamentx)
Tournamentx= pd.DataFrame(Tournamentx)
Tournamentx

,tournament_id,tournament_name,tournament_slug,tournament_category_name,tournament_category_slug,tournament_uniqueTournament_category_id,tournament_uniqueTournament_hasEventPlayerStatistics,tournament_model_id,tournament_isGroup,tournament_uniqueTournament_hasPerformanceGraphFeature
0,8,LaLiga,laliga,Spain,spain,32,False,36,False,False
1,9,Challenger Pro League,challenger-pro-league,Belgium,belgium,33,False,692,False,False
2,17,Premier League,premier-league,England,england,1,False,1,False,False
3,18,Championship,championship,England,england,1,False,2,False,False
4,20,Eliteserien,eliteserien,Norway,norway,5,False,5,False,False
...,...,...,...,...,...,...,...,...,...,...
99,11621,"Liga MX, Apertura",liga-mx-apertura,Mexico,mexico,12,False,28,False,False
100,11653,Primera División,primera-division,Chile,chile,49,False,64475,False,False
101,13362,"USL, League One",usl-league-one,USA,usa,26,False,71366,False,False
102,14688,Super Cup,super-cup,Egypt,egypt,305,False,22348,False,False


In [13]:
result = pd.merge(Teamdata,Seasonx , on="season_id")
result

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id_x,season_id,season_name,season_year,tournament_id_y
0,3,Wolverhampton,wolverhampton,Wolves,WOL,False,17,61627,Premier League 24/25,24/25,17
1,7,Crystal Palace,crystal-palace,Crystal Palace,CRY,False,17,61627,Premier League 24/25,24/25,17
2,14,Nottingham Forest,nottingham-forest,Forest,NFO,False,17,61627,Premier League 24/25,24/25,17
3,17,Manchester City,manchester-city,Man City,MCI,False,17,61627,Premier League 24/25,24/25,17
4,38,Chelsea,chelsea,Chelsea,CHE,False,17,61627,Premier League 24/25,24/25,17
...,...,...,...,...,...,...,...,...,...,...,...
654,405294,Central Valley Fuego,central-valley-fuego,CV Fuego,FUE,False,13362,57318,USL League One 2024,2024,13362
655,826802,Fiorentina U20,acf-fiorentina-u20-u20,Fiorentina U20,ACF,False,2292,64432,Primavera 1 24/25,24/25,2292
656,826810,Udinese U20,udinese-calcio-u20-u20,Udinese U20,UDI,False,2292,64432,Primavera 1 24/25,24/25,2292
657,826812,Genoa U20,genoa-cfc-u20-u20,Genoa U20,GEN,False,2292,64432,Primavera 1 24/25,24/25,2292
